In [1]:
import pandas as pd

data = pd.read_excel("Återbet.xlsx")

data = data[pd.to_numeric(data['OrderNo'], errors='coerce').notnull()]

data.loc[:, "OrderNo":"OrderNo"] = data.loc[:, "OrderNo":"OrderNo"].applymap(int)

data["PriceIncVat"] = data["PriceIncVat"].astype(int).replace('\.o', '', regex = True)

data = data[["ProductName", "OrderNo", "NoOfItems","PriceIncVat", "PriceExcVat", "Info"]]

data.dropna(subset=['ProductName'], inplace = True)

def fraktsätt(my_list, position):
    return my_list[position]

data["Info"] = data.Info.str.split(",").apply(fraktsätt, position=2)

data

,ProductName,OrderNo,NoOfItems,PriceIncVat,PriceExcVat,Info
0,Paketentré web Sommar,591909,2.0,280,224.0,Hemleverans
1,Rab-Åkpaketet låg web,591909,2.0,-100,-80.0,Hemleverans
2,Åkband web paket,591909,2.0,710,568.0,Hemleverans
3,Bonusåkband,591909,1.0,0,0.0,Hämta i park
4,Guldkortet ÅA,592470,1.0,1599,1279.2,Digitalt
...,...,...,...,...,...,...
107,Guldkortet ÅA,600198,1.0,1599,1279.2,Digitalt
108,Guldkortet ÅA,600549,1.0,1599,1279.2,Digitalt
109,Guldkortet ÅA,600549,1.0,1599,1279.2,Digitalt
110,Guldkortet web,600638,1.0,1699,1359.2,Hämta i park


In [2]:
data.NoOfItems[data.ProductName == 'Stora Gruppaketet'].sum()

30.0

In [3]:
def gruppaket_split(data):
    # Stora gruppaketet kostar 435 kr och är uppdelat så här:
    # Konto 2976: 380 kr (ref: 3054/110)
    # Konto 2976: -96 kr (ref: 3069/110)
    # Konto 2979: 80 kr (ref: 3923/110)
    # Konto 2610: 71 kr 

    try:
        antal = data.NoOfItems[data.ProductName == 'Stora Gruppaketet'].sum()
        split = [
        {'Konto':2976, 'Belopp': 380*antal, 'Referens': '3054/110'},
        {'Konto':2976, 'Belopp': -96*antal, 'Referens': '3069/110'},
        {'Konto':2979, 'Belopp': 80*antal, 'Referens': '3923/110'},
        {'Konto':2610, 'Belopp': 71*antal, 'Referens': ''},
        ] 
        split = pd.DataFrame(split)
        split["B"] = None
        split["C"]= None
        split["D"] = None
        split["E"] = None
    except:
        pass
    
    return split[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]

def guldkortet_split(data):
    try:
        antal = data.NoOfItems[data.ProductName == 'ÅA'].sum()
        split = [
        {'Konto': 2976, 'Belopp': 1359.2*antal, 'Referens': '3056/110'},
        {'Konto': 2976, 'Belopp': -80*antal, 'Referens': '3069/110'},
        {'Konto': 2610, 'Belopp': 319.8*antal, 'Referens': ''}
        ]
        split = pd.DataFrame(split)
        split["B"] = None
        split["C"]= None
        split["D"] = None
        split["E"] = None
    except:
        pass
    
    return split[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]
        


In [4]:
gruppaket_split(data)

,Konto,B,C,D,E,Belopp,Referens
0,2976,None,None,None,None,11400.0,3054/110
1,2976,None,None,None,None,-2880.0,3069/110
2,2979,None,None,None,None,2400.0,3923/110
3,2610,None,None,None,None,2130.0,


In [5]:
data["Moms"] = data["PriceIncVat"]-data["PriceExcVat"]

data["Referens"] = None

data["Belopp"] = None

data["Konto"] = "2976"

data

,ProductName,OrderNo,NoOfItems,PriceIncVat,PriceExcVat,Info,Moms,Referens,Belopp,Konto
0,Paketentré web Sommar,591909,2.0,280,224.0,Hemleverans,56.0,None,None,2976
1,Rab-Åkpaketet låg web,591909,2.0,-100,-80.0,Hemleverans,-20.0,None,None,2976
2,Åkband web paket,591909,2.0,710,568.0,Hemleverans,142.0,None,None,2976
3,Bonusåkband,591909,1.0,0,0.0,Hämta i park,0.0,None,None,2976
4,Guldkortet ÅA,592470,1.0,1599,1279.2,Digitalt,319.8,None,None,2976
...,...,...,...,...,...,...,...,...,...,...
107,Guldkortet ÅA,600198,1.0,1599,1279.2,Digitalt,319.8,None,None,2976
108,Guldkortet ÅA,600549,1.0,1599,1279.2,Digitalt,319.8,None,None,2976
109,Guldkortet ÅA,600549,1.0,1599,1279.2,Digitalt,319.8,None,None,2976
110,Guldkortet web,600638,1.0,1699,1359.2,Hämta i park,339.8,None,None,2976


In [6]:
data.loc[data['ProductName'].str.contains('Guldkortet web'), 'Referens'] = '3056/110'
data.loc[data['ProductName'].str.contains('Gröna Kortet'), 'Referens'] = '3018/110'
data.loc[data['ProductName'].str.contains('Åkband'), 'Referens'] = '3051/110'
data.loc[data['ProductName'].str.contains('Paketentré'), 'Referens'] = '3001/110'
data.loc[data['ProductName'].str.contains('House'), 'Referens'] = '3059/110'
data.loc[data['ProductName'].str.contains('Rab'), 'Referens'] = '3011/110'
data.loc[data['ProductName'].str.contains('Entré'), 'Referens'] = '3001/110'
data.loc[data['ProductName'].str.contains('Stora'), 'Referens'] = 'Stora Gruppaketet'
data.loc[data['ProductName'].str.contains('ÅA'), 'Referens'] = 'ÅA'

data.loc[data['ProductName'].str.contains('House'), 'Konto'] = "2978"
data.loc[data['ProductName'].str.contains('Present'), 'Konto'] = "2894"
data.loc[data['ProductName'].str.contains('Stora'), 'Konto'] = "1915"
data.loc[data['Info'].str.contains('Hämta'), 'Konto'] = "2813"

# Stora gruppaketet kostar 435 kr och är uppdelat så här:
# Konto 2976: 380 kr (ref: 3054/110)
# Konto 2976: -96 kr (ref: 3069/110)
# Konto 2979: 80 kr (ref: 3923/110)
# Konto 2610: 71 kr 

data

,ProductName,OrderNo,NoOfItems,PriceIncVat,PriceExcVat,Info,Moms,Referens,Belopp,Konto
0,Paketentré web Sommar,591909,2.0,280,224.0,Hemleverans,56.0,3001/110,None,2976
1,Rab-Åkpaketet låg web,591909,2.0,-100,-80.0,Hemleverans,-20.0,3011/110,None,2976
2,Åkband web paket,591909,2.0,710,568.0,Hemleverans,142.0,3051/110,None,2976
3,Bonusåkband,591909,1.0,0,0.0,Hämta i park,0.0,None,None,2813
4,Guldkortet ÅA,592470,1.0,1599,1279.2,Digitalt,319.8,ÅA,None,2976
...,...,...,...,...,...,...,...,...,...,...
107,Guldkortet ÅA,600198,1.0,1599,1279.2,Digitalt,319.8,ÅA,None,2976
108,Guldkortet ÅA,600549,1.0,1599,1279.2,Digitalt,319.8,ÅA,None,2976
109,Guldkortet ÅA,600549,1.0,1599,1279.2,Digitalt,319.8,ÅA,None,2976
110,Guldkortet web,600638,1.0,1699,1359.2,Hämta i park,339.8,3056/110,None,2813


In [7]:
grupp_paket_df = gruppaket_split(data)

In [8]:
guldkortet_df = guldkortet_split(data)

In [9]:
data["2610"] = None
data["2630"] = None

data.loc[data['Konto'].str.contains('2976'), '2610'] = data["Moms"]
data.loc[data['Konto'].str.contains('2978'), '2630'] = data["Moms"]

data.loc[data['Konto'].str.contains('2976'), 'Belopp'] = data["PriceExcVat"]
data.loc[data['Konto'].str.contains('2978'), 'Belopp'] = data["PriceExcVat"]
data.loc[data['Konto'].str.contains('2813'), 'Belopp'] = data["PriceIncVat"]
data.loc[data['Konto'].str.contains('1915'), 'Belopp'] = data["PriceIncVat"]
data.loc[data['Konto'].str.contains('2894'), 'Belopp'] = data["PriceIncVat"]

data

,ProductName,OrderNo,NoOfItems,PriceIncVat,PriceExcVat,Info,Moms,Referens,Belopp,Konto,2610,2630
0,Paketentré web Sommar,591909,2.0,280,224.0,Hemleverans,56.0,3001/110,224,2976,56,None
1,Rab-Åkpaketet låg web,591909,2.0,-100,-80.0,Hemleverans,-20.0,3011/110,-80,2976,-20,None
2,Åkband web paket,591909,2.0,710,568.0,Hemleverans,142.0,3051/110,568,2976,142,None
3,Bonusåkband,591909,1.0,0,0.0,Hämta i park,0.0,None,0,2813,None,None
4,Guldkortet ÅA,592470,1.0,1599,1279.2,Digitalt,319.8,ÅA,1279.2,2976,319.8,None
...,...,...,...,...,...,...,...,...,...,...,...,...
107,Guldkortet ÅA,600198,1.0,1599,1279.2,Digitalt,319.8,ÅA,1279.2,2976,319.8,None
108,Guldkortet ÅA,600549,1.0,1599,1279.2,Digitalt,319.8,ÅA,1279.2,2976,319.8,None
109,Guldkortet ÅA,600549,1.0,1599,1279.2,Digitalt,319.8,ÅA,1279.2,2976,319.8,None
110,Guldkortet web,600638,1.0,1699,1359.2,Hämta i park,339.8,3056/110,1699,2813,None,None


In [10]:
Moms_25 = data["2610"].sum()
Moms_6 = data["2630"].sum()


data.loc[-1] = [None, None, None, None, None, None, None, None, Moms_25,"2610",None,None]
data.loc[-2] = [None, None, None, None, None, None, None, None,Moms_6, "2630", None,None]

data

,ProductName,OrderNo,NoOfItems,PriceIncVat,PriceExcVat,Info,Moms,Referens,Belopp,Konto,2610,2630
0,Paketentré web Sommar,591909,2,280,224,Hemleverans,56,3001/110,224,2976,56,None
1,Rab-Åkpaketet låg web,591909,2,-100,-80,Hemleverans,-20,3011/110,-80,2976,-20,None
2,Åkband web paket,591909,2,710,568,Hemleverans,142,3051/110,568,2976,142,None
3,Bonusåkband,591909,1,0,0,Hämta i park,0,None,0,2813,None,None
4,Guldkortet ÅA,592470,1,1599,1279.2,Digitalt,319.8,ÅA,1279.2,2976,319.8,None
...,...,...,...,...,...,...,...,...,...,...,...,...
109,Guldkortet ÅA,600549,1,1599,1279.2,Digitalt,319.8,ÅA,1279.2,2976,319.8,None
110,Guldkortet web,600638,1,1699,1359.2,Hämta i park,339.8,3056/110,1699,2813,None,None
111,Presentkort 500,600691,1,500,500,Hemleverans,0,None,500,2894,None,None
-1,None,None,None,None,None,None,None,None,9905,2610,None,None


In [11]:
data["B"] = None
data["C"]= None
data["D"] = None
data["E"] = None


data = data[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]

data

,Konto,B,C,D,E,Belopp,Referens
0,2976,None,None,None,None,224,3001/110
1,2976,None,None,None,None,-80,3011/110
2,2976,None,None,None,None,568,3051/110
3,2813,None,None,None,None,0,None
4,2976,None,None,None,None,1279.2,ÅA
...,...,...,...,...,...,...,...
109,2976,None,None,None,None,1279.2,ÅA
110,2813,None,None,None,None,1699,3056/110
111,2894,None,None,None,None,500,None
-1,2610,None,None,None,None,9905,None


In [12]:
df =pd.concat([data,grupp_paket_df, guldkortet_df], ignore_index=True)

In [13]:
df = df[df.Referens != 'Stora Gruppaketet']

df = df[df.Referens != 'ÅA']

df

,Konto,B,C,D,E,Belopp,Referens
0,2976,None,None,None,None,224,3001/110
1,2976,None,None,None,None,-80,3011/110
2,2976,None,None,None,None,568,3051/110
3,2813,None,None,None,None,0,None
5,2813,None,None,None,None,0,None
...,...,...,...,...,...,...,...
116,2979,None,None,None,None,2400,3923/110
117,2610,None,None,None,None,2130,
118,2976,None,None,None,None,0,3056/110
119,2976,None,None,None,None,-0,3069/110


In [16]:
df = df[df.Belopp != 0]

df

,Konto,B,C,D,E,Belopp,Referens
0,2976,None,None,None,None,224,3001/110
1,2976,None,None,None,None,-80,3011/110
2,2976,None,None,None,None,568,3051/110
7,2976,None,None,None,None,104,3001/110
8,2976,None,None,None,None,1359.2,3056/110
...,...,...,...,...,...,...,...
113,2630,None,None,None,None,16.98,None
114,2976,None,None,None,None,11400,3054/110
115,2976,None,None,None,None,-2880,3069/110
116,2979,None,None,None,None,2400,3923/110


In [19]:
pd.set_option('display.max_rows', df.shape[0]+1)

df

,Konto,B,C,D,E,Belopp,Referens
0,2976,None,None,None,None,224,3001/110
1,2976,None,None,None,None,-80,3011/110
2,2976,None,None,None,None,568,3051/110
7,2976,None,None,None,None,104,3001/110
8,2976,None,None,None,None,1359.2,3056/110
9,2894,None,None,None,None,250,None
10,2976,None,None,None,None,248,3018/110
11,2894,None,None,None,None,250,None
12,2894,None,None,None,None,250,None
13,2976,None,None,None,None,248,3018/110
